# OPENCV官方文档:
https://docs.opencv.org/3.0-beta/modules/highgui/doc/user_interface.html

关于为什么要有0xFF:
- On some systems, waitKey() may return a value that encodes more than just the ASCII keycode. 
- On all systems, we can ensure that we extract just the ASCII keycode by reading the last byte from the return value like this: 
```python
keycode = cv2.waitKey(1)
if keycode != -1: 
    keycode &= 0xFF
```

In [6]:
# 测试
# 人脸检测
import cv2

cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height

faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_alt.xml')
eyeCascade = cv2.CascadeClassifier('Cascades/haarcascade_eye_tree_eyeglasses.xml')

while(True):
    ret, frame = cap.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # gray 表示输入 grayscale 图像。
    # scaleFactor 表示每个图像缩减的比例大小。
    # minNeighbors 表示每个备选矩形框具备的邻近数量。数字越大，假正类越少。
    # minSize 表示人脸识别的最小矩形大小。
    faces = faceCascade.detectMultiScale(
        gray, 
        scaleFactor=1.1,
        minNeighbors=2,
        minSize=(10, 10),
#         flags=cv2.CASCADE_SCALE_IMAGE  # 还不知道什么用
    )
    for (x,y,w,h) in faces:
        # 画矩形框
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        roi_color = frame[y:y+h, x:x+w]
        
    cv2.imshow('frame', frame)
    
    
    
    k = cv2.waitKey(10) & 0xff
    if k == ord('q'):  # press 'q' to quit  k == ord('q')
        break


cap.release()
cv2.destroyAllWindows()

        


In [3]:
import os
import imghdr

# 读取树梅派摄像头: MJPG-streamer方式
- 参考https://blog.csdn.net/m0_38106923/article/details/86562451#

## 启动数梅派
- 进入文件夹
`cd ./share/mjpg-streamer-master/mjpg-streamer-experimental`
- 指令启动普通USB摄像头指令：
`./mjpg_streamer -i "./input_uvc.so" -o "./output_http.so -w ./www"  `
- 启动树莓派专用摄像头RaspiCamera的指令：
`./mjpg_streamer -i "./input_raspicam.so" -o "./output_http.so -w ./www"`

## 进入页面
- 主页 http://192.168.1.101:8080/
- 静态 http://192.168.1.101:8080/?action=snapshot
- 动态 http://192.168.1.101:8080/?action=stream
- 或者 http://192.168.1.101:8080/javascript_simple.html

## opencv-python读取stream
- 参考1:读取stream https://blog.csdn.net/xiao__run/article/details/76342634
- 参考2:读取pdf中的图片 https://blog.csdn.net/u014267567/article/details/73252401

In [1]:
# 检测人脸

import cv2
import urllib
import numpy as np
import sys


def detect(img, cascade):
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 不需要转换为灰度图来检测
    faces = cascade.detectMultiScale(
        img, 
        scaleFactor=1.1,
        minNeighbors=2,
        minSize=(10, 10),
        # flags=cv2.CASCADE_SCALE_IMAGE  # 还不知道什么用
    )
    for (x,y,w,h) in faces:
        # 画矩形框
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        # roi_color = frame[y:y+h, x:x+w]  # 截取人脸
    return img


def get_steam():
    faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_alt.xml')

    host = "192.168.1.101"
#     if len(sys.argv)>1:
#         host = sys.argv[1]
    hoststr = 'http://' + host + '/?action=stream'
    print('Streaming ' + hoststr)
    print('Print "q" to quit')

    # stream=urllib.request.urlopen(hoststr)
    stream = urllib.request.urlopen("http://192.168.1.101:8080/?action=stream")

    bytes1 = b''
    while True:
        bytes1 += stream.read(1024)
        startmark = bytes1.find(b'\xff\xd8')
        endmark = bytes1.find(b'\xff\xd9')
        if startmark != -1 and endmark != -1:
            jpg = bytes1[startmark:endmark + 2]
            bytes1 = bytes1[endmark + 2:] #flags = 1 for color image 
            i = cv2.imdecode(np.fromstring(jpg, dtype=np.uint8),flags=1) # print i.shape

            img = detect(i, faceCascade)

            cv2.imshow("pi stream",img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
    #             exit(0)
                break

    stream.close()
    cv2.destroyAllWindows()

In [2]:
get_steam()

Streaming http://192.168.1.101/?action=stream
Print "q" to quit


/home/steven/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:46: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
